In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import nltk
import re
import requests
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import lxml
from io import StringIO
from collections import Counter
from sklearn.decomposition import PCA

import json
import os

import seaborn as sns
sns.set_style("darkgrid")
%pylab inline

#visualization
from IPython.display import Image
import pydotplus
from sklearn.tree import export_graphviz

# visualization
%matplotlib inline
import matplotlib.pyplot as plt

Populating the interactive namespace from numpy and matplotlib


### Read JSON for Yelp data

In [3]:
# will return a pandas df. provide location of JSON file
def convert_json(fileloc):
    yelp_list = [] # create an empty list
    with open(fileloc) as f:
        for line in f:
            yelp_list.append(json.loads(line))

        return pd.DataFrame(yelp_list)

In [41]:
tips = convert_json('data/dataset/tip.json')

In [5]:
business = convert_json('data/dataset/business.json')

In [12]:
checkin = convert_json('data/dataset/checkin.json')

In [42]:
review = convert_json('data/dataset/review.json')

In [4]:
user = convert_json('data/dataset/user.json')

In [ ]:
photos = convert_json('data/dataset/photos.json')

In [7]:
photos.head(15)

,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,...,cool,elite,fans,friends,funny,name,review_count,useful,user_id,yelping_since
0,3.80,5174,284,5174,5175,78,299,1435,7829,7397,...,16856,"[2014, 2016, 2013, 2011, 2012, 2015, 2010, 2017]",209,"[M19NwFwAXKRZzt8koF11hQ, QRcMZ8pJJBBZaKubHOoMD...",16605,Cin,272,17019,lsSiIjAKVl-QRxKjRErBeg,2010-07-13
1,3.94,1556,211,1556,1285,101,134,1295,162,2134,...,40110,"[2014, 2017, 2011, 2012, 2015, 2009, 2013, 200...",835,"[eoSSJzdprj3jxXyi94vDXg, QF0urZa-0bxga17ZeY-9l...",10882,Andrea,2559,83681,om5ZiponkpRqUNa3pVPiRg,2006-01-18
2,4.72,15,1,15,5,0,1,11,8,20,...,55,[],17,"[Oa84FFGBw1axX8O6uDkmqg, SRcWERSl4rhm-Bz9zN_J8...",4,Gabe,277,45,-lGwMGHMC_XihFJNKCJNRg,2014-10-31
3,3.76,9,0,9,1,0,1,4,1,11,...,4,"[2016, 2017]",11,"[96DJovjKAtExnyBZRfniPQ, Djczq-bRn761U4FW77NUt...",4,Leah,436,15,D-ydMTPGWXTVm4_jjp0k9g,2013-04-01
4,4.23,276,0,276,59,0,8,51,169,386,...,6006,"[2017, 2016]",49,"[iN0A6QIrEFYoSGHFaknh8Q, B2HDoWNIzLlon0IhS1cmD...",360,Juan,921,9152,PcvbBOCOcs6_suRDH7TSTg,2012-08-16
5,5.00,0,0,0,0,0,0,0,0,0,...,1,[],1,"[HanGVaZP_0IwhQdc0F5paQ, _v1_G2Fs56K8JTAzzItAM...",2,Charmaine,22,1,uXjR2GhCAYhqxVr21aC4vQ,2011-10-14
6,4.40,9,0,9,8,0,2,1,1,4,...,0,"[2017, 2016]",17,"[ZWD8UH1T7QXQr0Eq-mcWYg, ObSXluQeEYiOP9l9muFlG...",0,Victoria,91,0,N2arP_u4sMGLgOZhA6ZFoQ,2013-12-20
7,4.06,4,0,4,5,0,5,4,5,5,...,1,"[2016, 2015]",5,"[Puvuej6lzJ1JOEmtjG7V_Q, fq7CL1myWPYeH0d4bKtsI...",3,Erica,68,5,YHJIMK_zVH_VY6HCY6bYvg,2012-06-08
8,3.63,4,0,4,1,0,3,1,0,2,...,2,[],3,"[b8NTnxrhHCs6IPAKrzYL9w, vaNvEgtznFwSPDNkbM7hj...",11,Whitney,184,141,g3V76Ja0XgWS1rqx0gxL_A,2007-09-11
9,4.19,1,0,1,0,0,0,2,0,1,...,2,[],5,"[lM_HnyWHrpBsrgts3VnkxA, ynYGa-R1ZJf-tJOVlQeML...",6,Paul,48,0,VTL4Dep5p7XrrOtgnXdEIQ,2010-04-25


In [13]:
user_small = user[['user_id', 'elite', 'yelping_since']]

In [14]:
#user_small.columns = ['user_id', 'fans', 'friends', 'user_avg_stars', 'user_funny', 'user_cool', 'user_useful', 'elite', 'yelping_since']

In [15]:
user_small.head()

,user_id,elite,yelping_since
0,lsSiIjAKVl-QRxKjRErBeg,"[2014, 2016, 2013, 2011, 2012, 2015, 2010, 2017]",2010-07-13
1,om5ZiponkpRqUNa3pVPiRg,"[2014, 2017, 2011, 2012, 2015, 2009, 2013, 200...",2006-01-18
2,-lGwMGHMC_XihFJNKCJNRg,[],2014-10-31
3,D-ydMTPGWXTVm4_jjp0k9g,"[2016, 2017]",2013-04-01
4,PcvbBOCOcs6_suRDH7TSTg,"[2017, 2016]",2012-08-16


In [17]:
user_small.to_pickle('data/user_small.pkl')

In [8]:
user.columns

Index(['average_stars', 'compliment_cool', 'compliment_cute',
       'compliment_funny', 'compliment_hot', 'compliment_list',
       'compliment_more', 'compliment_note', 'compliment_photos',
       'compliment_plain', 'compliment_profile', 'compliment_writer', 'cool',
       'elite', 'fans', 'friends', 'funny', 'name', 'review_count', 'useful',
       'user_id', 'yelping_since'],
      dtype='object')

In [43]:
review.columns

Index(['business_id', 'cool', 'date', 'funny', 'review_id', 'stars', 'text',
       'useful', 'user_id'],
      dtype='object')

In [59]:
review['stars'] = review['stars'].astype('category')

In [60]:
review.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4736897 entries, 0 to 4736896
Data columns (total 9 columns):
business_id    object
cool           int64
date           object
funny          int64
review_id      object
stars          category
text           object
useful         int64
user_id        object
dtypes: category(1), int64(3), object(5)
memory usage: 293.6+ MB


In [61]:
review1 = review.iloc[0:1000000, ]
review2 = review.iloc[1000000 : 2000000, ]
review3 = review.iloc[2000000 : 3000000, ]
review4 = review.iloc[3000000 : 4000000, ]
review5 = review.iloc[4000000 :, ]


In [62]:
 review1.to_pickle('data/review1.pkl')

In [63]:
review2.to_pickle('data/review2.pkl')

In [64]:
review3.to_pickle('data/review3.pkl')

In [65]:
review4.to_pickle('data/review4.pkl')

In [66]:
review5.to_pickle('data/review5.pkl')

In [ ]:
review5['MA'] = review5['pop'].rolling(window=5,center=False).mean()

In [2]:
review1 = np.load('data/review1.pkl')
review1.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,uYHaNptLzDLoV_JZ_MuzUA,0,2016-07-12,0,VfBHSwC5Vz_pbFluy07i9Q,5,My girlfriend and I stayed here for 3 nights a...,0,cjpdDjZyprfyDG3RlkVG3w
1,uYHaNptLzDLoV_JZ_MuzUA,0,2016-10-02,0,3zRpneRKDsOPq92tq7ybAA,3,If you need an inexpensive place to stay for a...,0,bjTcT8Ty4cJZhEOEo01FGA
2,uYHaNptLzDLoV_JZ_MuzUA,0,2015-09-17,0,ne5WhI1jUFOcRn-b-gAzHA,3,Mittlerweile gibt es in Edinburgh zwei Ableger...,0,AXgRULmWcME7J6Ix3I--ww
3,uYHaNptLzDLoV_JZ_MuzUA,0,2016-08-21,0,llmdwOgDReucVoWEry61Lw,4,Location is everything and this hotel has it! ...,0,oU2SSOmsp_A8JYI7Z2JJ5w
4,uYHaNptLzDLoV_JZ_MuzUA,0,2013-11-20,0,DuffS87NaSMDmIfluvT83g,5,gute lage im stadtzentrum. shoppingmeile und s...,0,0xtbPEna2Kei11vsU-U2Mw


In [ ]:
pivoted_df = pd.pivot_table(review1, index='date', columns='review_id', values='useful')
#pivoted_df

In [ ]:
average_fruits = pivoted_df.rolling(window=3).mean().stack() 


In [3]:
review1.groupby('review_id').useful.cumsum()

0         0
1         0
2         0
3         0
4         0
5         0
6         0
7         0
8         2
9         2
10        1
11        1
12        1
13        1
14        1
15        1
16        0
17        1
18        2
19        1
20        0
21        0
22        1
23        0
24        0
25        1
26        0
27        1
28        1
29        0
         ..
999970    0
999971    0
999972    0
999973    0
999974    5
999975    4
999976    1
999977    0
999978    0
999979    0
999980    0
999981    0
999982    3
999983    0
999984    0
999985    2
999986    0
999987    1
999988    0
999989    0
999990    1
999991    0
999992    0
999993    0
999994    0
999995    1
999996    0
999997    2
999998    4
999999    3
Name: useful, Length: 1000000, dtype: int64

### Load data

In [7]:
review.shape

(4736897, 9)

In [8]:
review_small = review.iloc[:100000, :]

In [9]:
review_small.shape

(100000, 9)

In [10]:
review_small.to_pickle('data/review_small.pkl')

#### Pickle some files

In [14]:
# tips.to_pickle('data/tips.pkl')

In [15]:
# business.to_pickle('data/business.pkl')

In [16]:
#checkin.to_pickle('data/checkin.pkl')

In [ ]:
#review_small.to_pickle('data/review_small.pkl')

In [ ]:
import pickle
import os.path

In [ ]:
file_path = "data/review.pkl"
n_bytes = 2**31
max_bytes = 2**31 - 1
data = bytearray(n_bytes)

## write
bytes_out = pickle.dumps(review)
with open(file_path, 'wb') as f_out:
    for idx in range(0, n_bytes, max_bytes):
        f_out.write(bytes_out[idx:idx+max_bytes])

## read
# bytes_in = bytearray(0)
# input_size = os.path.getsize(file_path)
# with open(file_path, 'rb') as f_in:
#     for _ in range(0, input_size, max_bytes):
#         bytes_in += f_in.read(max_bytes)
# data2 = pickle.loads(bytes_in)

# assert(data == data2)



### Start Analysis

In [4]:
review_small = np.load("data/review_small.pkl")

In [5]:
review_small.head(30)

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,uYHaNptLzDLoV_JZ_MuzUA,0,2016-07-12,0,VfBHSwC5Vz_pbFluy07i9Q,5,My girlfriend and I stayed here for 3 nights a...,0,cjpdDjZyprfyDG3RlkVG3w
1,uYHaNptLzDLoV_JZ_MuzUA,0,2016-10-02,0,3zRpneRKDsOPq92tq7ybAA,3,If you need an inexpensive place to stay for a...,0,bjTcT8Ty4cJZhEOEo01FGA
2,uYHaNptLzDLoV_JZ_MuzUA,0,2015-09-17,0,ne5WhI1jUFOcRn-b-gAzHA,3,Mittlerweile gibt es in Edinburgh zwei Ableger...,0,AXgRULmWcME7J6Ix3I--ww
3,uYHaNptLzDLoV_JZ_MuzUA,0,2016-08-21,0,llmdwOgDReucVoWEry61Lw,4,Location is everything and this hotel has it! ...,0,oU2SSOmsp_A8JYI7Z2JJ5w
4,uYHaNptLzDLoV_JZ_MuzUA,0,2013-11-20,0,DuffS87NaSMDmIfluvT83g,5,gute lage im stadtzentrum. shoppingmeile und s...,0,0xtbPEna2Kei11vsU-U2Mw
5,uYHaNptLzDLoV_JZ_MuzUA,0,2016-06-05,0,GvLmUkjUrOyFH8KFnmT1uw,5,Erstklassige Lage. Dazu ist alles geschrieben ...,0,rW8q706dz5-NnXDzMwVkiw
6,uYHaNptLzDLoV_JZ_MuzUA,0,2015-02-21,0,lGEl24NGj2HVBJrodeXcjg,4,"Beautiful space, great location, staff rock. T...",0,yx8vNXUL0D0HS8rUIC7AFA
7,uYHaNptLzDLoV_JZ_MuzUA,0,2013-07-07,0,cUgvEy5wj7zYE68v1BzzVg,4,This is a fairly new property I think. It is a...,0,zXnH6W74FAJQ7q7b-NuBsA
8,uYHaNptLzDLoV_JZ_MuzUA,2,2013-04-27,0,FSB_BnvysBgH3JYrbFNcgw,4,First time at this group of hotels. Pretty new...,2,c5yp5hxwC1N98MjbV2LyWQ
9,uYHaNptLzDLoV_JZ_MuzUA,0,2015-04-13,0,dhl3ZW9aAEX_T7_um5tfaQ,4,Location location location! \n\nMotel One is j...,2,xJisL5w4wOgiYLokGMT_IA


In [31]:
review_small.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
business_id    100000 non-null object
cool           100000 non-null int64
date           100000 non-null object
funny          100000 non-null int64
review_id      100000 non-null object
stars          100000 non-null int64
text           100000 non-null object
useful         100000 non-null int64
user_id        100000 non-null object
dtypes: int64(4), object(5)
memory usage: 6.9+ MB


In [32]:
review_small['stars'] = review_small['stars'].astype('category')


In [33]:
review_small.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 9 columns):
business_id    100000 non-null object
cool           100000 non-null int64
date           100000 non-null object
funny          100000 non-null int64
review_id      100000 non-null object
stars          100000 non-null category
text           100000 non-null object
useful         100000 non-null int64
user_id        100000 non-null object
dtypes: category(1), int64(3), object(5)
memory usage: 6.2+ MB


In [6]:
business = np.load("data/business.pkl")
business.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,691 Richmond Rd,"{'RestaurantsPriceRange2': 2, 'BusinessParking...",YDf95gJZaq05wvo7hTQbbQ,"[Shopping, Shopping Centers]",Richmond Heights,"{'Monday': '10:00-21:00', 'Tuesday': '10:00-21...",1,41.541716,-81.493116,Richmond Town Square,,44143,17,2.0,OH
1,2824 Milton Rd,"{'GoodForMeal': {'dessert': False, 'latenight'...",mLwM-h2YhXl2NCgdS84_Bw,"[Food, Soul Food, Convenience Stores, Restaura...",Charlotte,"{'Monday': '10:00-22:00', 'Tuesday': '10:00-22...",0,35.236870,-80.741976,South Florida Style Chicken & Ribs,Eastland,28215,4,4.5,NC
2,337 Danforth Avenue,"{'BusinessParking': {'garage': False, 'street'...",v2WhjAB3PIBA8J8VxG3wEg,"[Food, Coffee & Tea]",Toronto,"{'Monday': '10:00-19:00', 'Tuesday': '10:00-19...",0,43.677126,-79.353285,The Tea Emporium,Riverdale,M4K 1N7,7,4.5,ON
3,"7702 E Doubletree Ranch Rd, Ste 300",{},CVtCbSB1zUcUWg-9TNGTuQ,"[Professional Services, Matchmakers]",Scottsdale,"{'Friday': '9:00-17:00', 'Tuesday': '9:00-17:0...",1,33.565082,-111.916400,TRUmatch,,85258,3,3.0,AZ
4,4719 N 20Th St,"{'RestaurantsTableService': False, 'GoodForMea...",duHFBe87uNSXImQmvBh87Q,"[Sandwiches, Restaurants]",Phoenix,{},0,33.505928,-112.038847,Blimpie,,85016,10,4.5,AZ


In [36]:
bus_small = business[["name", "business_id", "categories", "attributes", 'stars', 'review_count']]


bus_small['categories2'] = bus_small['categories'].apply(lambda x: ', '.join(x)).str.replace(',', '')
# bus_small['categories2'] = bus_small['categories2'].str.replace(',', '')
  

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [37]:
bus_small.head()

,name,business_id,categories,attributes,stars,review_count,categories2
0,Richmond Town Square,YDf95gJZaq05wvo7hTQbbQ,"[Shopping, Shopping Centers]","{'RestaurantsPriceRange2': 2, 'BusinessParking...",2.0,17,Shopping Shopping Centers
1,South Florida Style Chicken & Ribs,mLwM-h2YhXl2NCgdS84_Bw,"[Food, Soul Food, Convenience Stores, Restaura...","{'GoodForMeal': {'dessert': False, 'latenight'...",4.5,4,Food Soul Food Convenience Stores Restaurants
2,The Tea Emporium,v2WhjAB3PIBA8J8VxG3wEg,"[Food, Coffee & Tea]","{'BusinessParking': {'garage': False, 'street'...",4.5,7,Food Coffee & Tea
3,TRUmatch,CVtCbSB1zUcUWg-9TNGTuQ,"[Professional Services, Matchmakers]",{},3.0,3,Professional Services Matchmakers
4,Blimpie,duHFBe87uNSXImQmvBh87Q,"[Sandwiches, Restaurants]","{'RestaurantsTableService': False, 'GoodForMea...",4.5,10,Sandwiches Restaurants


In [29]:
# convert several Dummy Flags if category contains this description
for category in ['Sushi', 'Burger', 'Pizza', 'Coffee']:
    bus_small.loc[:, category] = bus_small.categories2.str.contains(category).astype(int)


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:517: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/pandas/core/indexing.py:337: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[key] = _infer_fill_value(value)


In [30]:
bus_small.head()

,name,categories,attributes,categories2,Sushi,Burger,Pizza,Coffee
0,Richmond Town Square,"[Shopping, Shopping Centers]","{'RestaurantsPriceRange2': 2, 'BusinessParking...",Shopping Shopping Centers,0,0,0,0
1,South Florida Style Chicken & Ribs,"[Food, Soul Food, Convenience Stores, Restaura...","{'GoodForMeal': {'dessert': False, 'latenight'...",Food Soul Food Convenience Stores Restaurants,0,0,0,0
2,The Tea Emporium,"[Food, Coffee & Tea]","{'BusinessParking': {'garage': False, 'street'...",Food Coffee & Tea,0,0,0,1
3,TRUmatch,"[Professional Services, Matchmakers]",{},Professional Services Matchmakers,0,0,0,0
4,Blimpie,"[Sandwiches, Restaurants]","{'RestaurantsTableService': False, 'GoodForMea...",Sandwiches Restaurants,0,0,0,0


In [35]:
bus_small.head()

,name,categories,attributes,categories2,flag
0,Richmond Town Square,"[Shopping, Shopping Centers]","{'RestaurantsPriceRange2': 2, 'BusinessParking...",Shopping Shopping Centers,0
1,South Florida Style Chicken & Ribs,"[Food, Soul Food, Convenience Stores, Restaura...","{'GoodForMeal': {'dessert': False, 'latenight'...",Food Soul Food Convenience Stores Restaurants,0
2,The Tea Emporium,"[Food, Coffee & Tea]","{'BusinessParking': {'garage': False, 'street'...",Food Coffee & Tea,0
3,TRUmatch,"[Professional Services, Matchmakers]",{},Professional Services Matchmakers,0
4,Blimpie,"[Sandwiches, Restaurants]","{'RestaurantsTableService': False, 'GoodForMea...",Sandwiches Restaurants,0


In [38]:
bus_small.flag.value_counts()

0    154806
1      1833
Name: flag, dtype: int64

In [36]:
bus_small[bus_small.flag == 1]

,name,categories,attributes,categories2,flag
53,Oishi Sushi,"[Asian Fusion, Restaurants, Sushi Bars]","{'RestaurantsTableService': True, 'GoodForMeal...",Asian Fusion Restaurants Sushi Bars,1
178,Sushi Hoshimi,"[Restaurants, Japanese, Sushi Bars]","{'RestaurantsTableService': True, 'GoodForMeal...",Restaurants Japanese Sushi Bars,1
182,Sushi Osaka,"[Sushi Bars, Restaurants, Japanese, Korean]","{'RestaurantsTakeOut': True, 'WiFi': 'free', '...",Sushi Bars Restaurants Japanese Korean,1
196,Sushi Shop,"[Restaurants, Sushi Bars]","{'HasTV': False, 'GoodForKids': True, 'NoiseLe...",Restaurants Sushi Bars,1
330,Oban Sushi,"[Sushi Bars, Restaurants]",{},Sushi Bars Restaurants,1
349,Heart Sushi,"[Restaurants, Sushi Bars, Japanese]","{'Alcohol': 'beer_and_wine', 'GoodForKids': Tr...",Restaurants Sushi Bars Japanese,1
451,Ilpunji Sushi House,"[Restaurants, Sushi Bars]","{'RestaurantsTableService': True, 'GoodForMeal...",Restaurants Sushi Bars,1
509,Shinji,"[Restaurants, Japanese, Sushi Bars]","{'RestaurantsTableService': True, 'GoodForMeal...",Restaurants Japanese Sushi Bars,1
626,Fuji Sushi & Grill,"[Sushi Bars, Restaurants]","{'RestaurantsTableService': True, 'GoodForMeal...",Sushi Bars Restaurants,1
640,Omoide Sushi & Noodle Bar,"[Noodles, Sushi Bars, Restaurants, Tapas/Small...","{'RestaurantsTableService': True, 'GoodForMeal...",Noodles Sushi Bars Restaurants Tapas/Small Pla...,1


In [12]:
review_small.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,uYHaNptLzDLoV_JZ_MuzUA,0,2016-07-12,0,VfBHSwC5Vz_pbFluy07i9Q,5,My girlfriend and I stayed here for 3 nights a...,0,cjpdDjZyprfyDG3RlkVG3w
1,uYHaNptLzDLoV_JZ_MuzUA,0,2016-10-02,0,3zRpneRKDsOPq92tq7ybAA,3,If you need an inexpensive place to stay for a...,0,bjTcT8Ty4cJZhEOEo01FGA
2,uYHaNptLzDLoV_JZ_MuzUA,0,2015-09-17,0,ne5WhI1jUFOcRn-b-gAzHA,3,Mittlerweile gibt es in Edinburgh zwei Ableger...,0,AXgRULmWcME7J6Ix3I--ww
3,uYHaNptLzDLoV_JZ_MuzUA,0,2016-08-21,0,llmdwOgDReucVoWEry61Lw,4,Location is everything and this hotel has it! ...,0,oU2SSOmsp_A8JYI7Z2JJ5w
4,uYHaNptLzDLoV_JZ_MuzUA,0,2013-11-20,0,DuffS87NaSMDmIfluvT83g,5,gute lage im stadtzentrum. shoppingmeile und s...,0,0xtbPEna2Kei11vsU-U2Mw


In [27]:
review.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,uYHaNptLzDLoV_JZ_MuzUA,0,2016-07-12,0,VfBHSwC5Vz_pbFluy07i9Q,5,My girlfriend and I stayed here for 3 nights a...,0,cjpdDjZyprfyDG3RlkVG3w
1,uYHaNptLzDLoV_JZ_MuzUA,0,2016-10-02,0,3zRpneRKDsOPq92tq7ybAA,3,If you need an inexpensive place to stay for a...,0,bjTcT8Ty4cJZhEOEo01FGA
2,uYHaNptLzDLoV_JZ_MuzUA,0,2015-09-17,0,ne5WhI1jUFOcRn-b-gAzHA,3,Mittlerweile gibt es in Edinburgh zwei Ableger...,0,AXgRULmWcME7J6Ix3I--ww
3,uYHaNptLzDLoV_JZ_MuzUA,0,2016-08-21,0,llmdwOgDReucVoWEry61Lw,4,Location is everything and this hotel has it! ...,0,oU2SSOmsp_A8JYI7Z2JJ5w
4,uYHaNptLzDLoV_JZ_MuzUA,0,2013-11-20,0,DuffS87NaSMDmIfluvT83g,5,gute lage im stadtzentrum. shoppingmeile und s...,0,0xtbPEna2Kei11vsU-U2Mw


In [21]:
checkin.head(20)

,business_id,time
0,7KPBkxAOEtb3QeIL9PEErg,"{'Thursday': {'21:00': 4, '1:00': 1, '4:00': 1..."
1,kREVIrSBbtqBhIYkTccQUg,"{'Monday': {'13:00': 1}, 'Thursday': {'20:00':..."
2,tJRDll5yqpZwehenzE2cSg,"{'Monday': {'12:00': 1, '1:00': 1}, 'Saturday'..."
3,nhZ1HGWD8lMErdn3FuWuTQ,"{'Sunday': {'18:00': 1, '17:00': 2, '22:00': 1..."
4,vDoXZGE7p6xAkKQ0XQPvoA,"{'Thursday': {'15:00': 1}, 'Saturday': {'23:00..."
5,8bY6M2yiWOF2ilfmGS34Fw,"{'Saturday': {'11:00': 1}, 'Thursday': {'14:00..."
6,zNVot5_XHsxwfKdiFjk_aA,"{'Wednesday': {'11:00': 1, '23:00': 1}, 'Sunda..."
7,BH0K6SxPnY3wTRB7tA27hQ,"{'Thursday': {'22:00': 1, '23:00': 1, '19:00':..."
8,dPxL6P_Gp74CqSPuL59Srg,"{'Wednesday': {'14:00': 2, '16:00': 1, '2:00':..."
9,aZbbt_RPKV8vUY7GWiEEyw,{'Thursday': {'18:00': 1}}


In [18]:
business.head()

,address,attributes,business_id,categories,city,hours,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,691 Richmond Rd,"{'RestaurantsPriceRange2': 2, 'BusinessParking...",YDf95gJZaq05wvo7hTQbbQ,"[Shopping, Shopping Centers]",Richmond Heights,"{'Monday': '10:00-21:00', 'Tuesday': '10:00-21...",1,41.541716,-81.493116,Richmond Town Square,,44143,17,2.0,OH
1,2824 Milton Rd,"{'GoodForMeal': {'dessert': False, 'latenight'...",mLwM-h2YhXl2NCgdS84_Bw,"[Food, Soul Food, Convenience Stores, Restaura...",Charlotte,"{'Monday': '10:00-22:00', 'Tuesday': '10:00-22...",0,35.236870,-80.741976,South Florida Style Chicken & Ribs,Eastland,28215,4,4.5,NC
2,337 Danforth Avenue,"{'BusinessParking': {'garage': False, 'street'...",v2WhjAB3PIBA8J8VxG3wEg,"[Food, Coffee & Tea]",Toronto,"{'Monday': '10:00-19:00', 'Tuesday': '10:00-19...",0,43.677126,-79.353285,The Tea Emporium,Riverdale,M4K 1N7,7,4.5,ON
3,"7702 E Doubletree Ranch Rd, Ste 300",{},CVtCbSB1zUcUWg-9TNGTuQ,"[Professional Services, Matchmakers]",Scottsdale,"{'Friday': '9:00-17:00', 'Tuesday': '9:00-17:0...",1,33.565082,-111.916400,TRUmatch,,85258,3,3.0,AZ
4,4719 N 20Th St,"{'RestaurantsTableService': False, 'GoodForMea...",duHFBe87uNSXImQmvBh87Q,"[Sandwiches, Restaurants]",Phoenix,{},0,33.505928,-112.038847,Blimpie,,85016,10,4.5,AZ


In [66]:
business.city.value_counts()

Las Vegas                 24768
Phoenix                   15656
Toronto                   15483
Charlotte                  7557
Scottsdale                 7510
Pittsburgh                 5688
Montréal                   5175
Mesa                       5146
Henderson                  4130
Tempe                      3949
Chandler                   3649
Edinburgh                  3625
Cleveland                  2979
Madison                    2891
Glendale                   2841
Gilbert                    2835
Mississauga                2316
Stuttgart                  1975
Peoria                     1527
Markham                    1390
North Las Vegas            1254
Champaign                  1084
North York                  944
Surprise                    934
Scarborough                 870
Richmond Hill               770
Concord                     726
Goodyear                    703
Brampton                    690
Vaughan                     650
                          ...  
St-Laure

In [67]:
business.state.value_counts()

AZ     47376
NV     30571
ON     26520
NC     11299
OH     10930
PA      8916
QC      7273
WI      4190
EDH     3561
BW      3071
IL      1667
SC       583
MLN      185
HLD      175
FIF       73
ELN       42
NYK       42
WLN       36
C         32
NY        15
NI        11
ESX       11
ST         9
CA         6
01         6
SCB        5
BY         4
WA         3
ABE        2
XGL        2
AL         2
NTH        2
TAM        1
KHL        1
STG        1
FL         1
FLN        1
SL         1
VT         1
DE         1
ZET        1
75         1
NE         1
HH         1
GLG        1
NLK        1
WHT        1
RCC        1
FAL        1
3          1
PKN        1
Name: state, dtype: int64

In [12]:
yelp_data[0:2]

[{'business_id': 'tJRDll5yqpZwehenzE2cSg',
  'date': '2012-07-15',
  'likes': 0,
  'text': 'Get here early enough to have dinner.',
  'user_id': 'zcTZk7OG8ovAmh_fenH21g'},
 {'business_id': 'jH19V2I9fIslnNhDzPmdkA',
  'date': '2015-08-12',
  'likes': 0,
  'text': 'Great breakfast large portions and friendly waitress. I highly recommend it',
  'user_id': 'ZcLKXikTHYOnYt5VYRO5sg'}]

In [2]:
import numpy as np

In [4]:
review1 = np.load(("data/review1.pkl"))

In [4]:
review1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 9 columns):
business_id    1000000 non-null object
cool           1000000 non-null int64
date           1000000 non-null object
funny          1000000 non-null int64
review_id      1000000 non-null object
stars          1000000 non-null category
text           1000000 non-null object
useful         1000000 non-null int64
user_id        1000000 non-null object
dtypes: category(1), int64(3), object(5)
memory usage: 62.0+ MB
